> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 2.0`: scoring mode demo
One of the more common running modes of `REINVENT 2.0` in a project setting is the *scoring mode*. As is described in detail in the *reinforcement learning* notebooks, an agent can learn (iteratively) to maximise the reward given by a scoring function. Recall that this function can be a (complex) combination of scoring components and it is not always straightforward to predict how a given compound will score. A common use case is to have a couple of compounds at the beginning of the project that are considered "good" and one would like to ensure that they score highly. If not, there is still the possibility to change the scoring function accordingly.

For this scenario, one can subject a file with molecules in the `SMILES` format to a scoring run, which is illustrated by this notebook.

To proceed, please update the following code block such that it reflects your system's installation and execute it.

In [1]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/Projects/Publications/2020/2020-04_REINVENT_2.0/Reinvent")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent_shared.v2.1")
output_dir = os.path.expanduser("~/Desktop/REINVENT_scoring_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

## Setting up the configuration
`REINVENT` has an entry point that loads a specified `JSON` file on startup. `JSON` is a low-level data format that allows to specify a fairly large number of parameters in a cascading fashion very quickly. The parameters are structured into *blocks* which can in turn contain blocks or simple values, such as *True* or *False*, strings and numbers. In this tutorial, we will go through the different blocks step-by-step, explaining their purpose and potential values for given parameters. Note, that while we will write out the configuration as a `JSON` file in the end, in `python` we handle the same information as a simple `dict`.

In [2]:
# initialize the dictionary
configuration = {
    "version": 2,                          # we are going to use REINVENT's newest release
    "run_type": "scoring"                  # other run types: "sampling", "validation",
                                           #                  "transfer_learning",
                                           #                  "reinforcement_learning" and
                                           #                  "create_model"
}

In [3]:
# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "http://127.0.0.1",          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                  # either to local logging or use a remote REST-interface
    "logging_path": os.path.join(output_dir, "progress.log"), # where the run's output is stored
    "job_name": "Scoring mode demo",       # set an arbitrary job name for identification
    "job_id": "demo"                       # only relevant if "recipient" is set to "remote"
}

In contrast to the *reinforcement learning* notebooks, we do not need to specify any prior or agent files, as we are only interested in the scores and no training is preformed at all. For each of our input molecules (see print-out below), we will calculate all individual components and the final score.

In [4]:
# set path to dummy input file
input_SMILES_path = os.path.join(ipynb, "data", "smiles.smi")

# add the "parameters" block
configuration["parameters"] = {}

# set all "reinforcement learning"-specific run parameters
configuration["parameters"]["scoring"] = {
    "input": input_SMILES_path
}

# print file contents for illustrative purposes
!head -n 15 {input_SMILES_path}

O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N


### Define the scoring function
Now all that remains to be done is the most tricky step: define a scoring function that allows to identify promising suggestions and discard molecules that are of no interest to the project. It is not necessarily better to build a very complex scoring function (on the contrary it can make it hard for the agent to find appropriate solutions). Always bear in mind that there is a post-processing step at the end, in which you will be able to discard molecules either by eye-inspection or by applying further (probably more expensive) methods you have not used in the reinforcement learning loop. The following example will include fair share of the available scoring function components (added one-by-one), but this is for illustrative purposes only.

##### Score transformation
Before we start, there is one more topic requiring some explanation: *score transformations*. Remember that every component returns a value between '0' and '1' (higher values meaning "better") and all scores together are combined into a *total score* for a given compound (also between '0' and '1'). This is key, as the agent will try to generate molecules with ever increasing scores over the course of training, i.e. the numerical value "guides" the agent. However, some components might not naturally return values between '0' or '1' or they might represent the opposite, i.e. '0' being "good" rather than "bad". This is component-specific and to make it as flexible as possible, we include the specification of a score transformation for each component. We support multiple different functions (`sigmoid`, `reverse_sigmoid` and so on) which have different parameters to allow tweaking them to the desired result. For more details and to see how different parameter values affect the result, we refer to the dedicated notebook which is also part of this repository.

In [5]:
# prepare the scoring function definition and add at the end
scoring_function = {
    "name": "custom_product",              # this is our default one (alternative: "custom_sum")

    # the "parameters" list holds the individual components
    "parameters": [

    # add component: an activity model
    {
        "component_type": "predictive_property", # this is a scikit-learn model, returning
                                                 # activity values
        "name": "Regression model",        # arbitrary name for the component
        "weight": 2,                       # the weight ("importance") of the component (default: 1)
        "model_path": os.path.join(ipynb_path, "models/Aurora_model.pkl"),   # absolute model path
        "smiles": [],                            # list of SMILES (not required for this component)
        "specific_parameters": {
            "transformation_type": "sigmoid",  # see description above
            "high": 9,                         # parameter for sigmoid transformation
            "low": 4,                          # parameter for sigmoid transformation
            "k": 0.25,                         # parameter for sigmoid transformation
            "scikit": "regression",            # model can be "regression" or "classification"
            "transformation": True,            # enable the transformation
            "descriptor_type": "ecfp_counts",  # sets the input descriptor for this model
            "size": 2048,                      # parameter of descriptor type
            "radius": 3,                       # parameter of descriptor type
            "use_counts": True,                # parameter of descriptor type
            "use_features": True               # parameter of descriptor type
        }
    },

    # add component: enforce the match to a given substructure
    {
        "component_type": "matching_substructure", 
        "name": "Matching substructure",       # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "model_path": None,                    # not required; note, this is "null" in JSON
        "smiles": ["c1ccccc1CC"],              # a match with this substructure is required
        "specific_parameters": None            # not required; note, this is "null" in JSON
    },

    # add component: enforce to NOT match a given substructure
    {
        "component_type": "custom_alerts",
        "name": "Custom alerts",               # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "model_path": None,                    # not required; note, this is "null" in JSON
        "smiles": [                            # specify the substructures (as list) to penalize
            "[*;r8]",
            "[*;r9]",
            "[*;r10]",
            "[*;r11]",
            "[*;r12]",
            "[*;r13]",
            "[*;r14]",
            "[*;r15]",
            "[*;r16]",
            "[*;r17]",
            "[#8][#8]",
            "[#6;+]",
            "[#16][#16]",
            "[#7;!n][S;!$(S(=O)=O)]",
            "[#7;!n][#7;!n]",
            "C#C",
            "C(=[O,S])[O,S]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
        ],
        "specific_parameters": None            # not required; note, this is "null" in JSON
    },

    # add component: calculate the QED drug-likeness score (using RDkit)
    {
        "component_type": "qed_score",
        "name": "QED Score",                   # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "model_path": None,                    # not required; note, this is "null" in JSON
        "smiles": [],                          # not required
        "specific_parameters": None            # not required; note, this is "null" in JSON
    }]
}
configuration["parameters"]["scoring_function"] = scoring_function

Note, that this definition is exactly the same as we have used for the *reinforcement learning* demo. Actually, the differences in the `JSON`s are only marginal: **(a)** replace the `run_type`, **(b)** remove the `results` path and `logging_frequency` in the header (as we will not *generate* any molecules) and **(c)** update the block with the path to the input `SMILES` file.

We now have successfully filled the dictionary and will write it out as a `JSON` file in the output directory. Please have a look at the file before proceeding in order to see how the paths have been inserted where required and the `dict` -> `JSON` translations (e.g. `True` to `true`) have taken place.

In [6]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "scoring_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

## Run `REINVENT`
Now it is time to execute `REINVENT` locally. As we will not update any weights, execution should be very fast. The result will be a `CSV` file in the logging directory, showing the `SMILES` and the individual components for each of them.

The command-line execution looks like this:
```
# activate envionment
conda activate reinvent_shared.v2.1

# execute REINVENT
python <your_path>/input.py <config>.json
```

In [7]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!python {reinvent_dir}/input.py {configuration_JSON_path}

In [8]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

As shown below, the `CSV` always at least three columns: `smiles`, `total_score` and (at the very end) `valid`, providing the molecules in `SMILES` format, the combined score and a flat indicating, whether or not these molecules are valid in the sense that you can build an `RDkit` molecule object from it or not. In between, all the the scoring components we have specified are given with the names we have used in our `JSON`: `Regression model`, `Matching substructure`, `Custom alerts` and `QED Score`.

In [9]:
# print the resulting CSV file
!head -n 15 {output_dir}/progress.log/scored_smiles.csv

smiles,total_score,Regression model,Matching substructure,Custom alerts,QED Score,valid
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N,0.20701039,0.3067730665206909,0.5,1.0,0.7541053295135498,1
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N,0.20701039,0.3067730665206909,0.5,1.0,0.7541053295135498,1
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N,0.20701039,0.3067730665206909,0.5,1.0,0.7541053295135498,1
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N,0.20701039,0.3067730665206909,0.5,1.0,0.7541053295135498,1
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N,0.20701039,0.3067730665206909,0.5,1.0,0.7541053295135498,1
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N,0.20701039,0.3067730665206909,0.5,1.0,0.7541053295135498,1
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N,0.20701039,0.3067730665206909,0.5,1.0,0.7541053295135498,1
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N,0.20701039,0.3067730665206909,0.5,1.0,0.7541053295135498,1
O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N